In [1]:
import torch
import torch.nn as nn
import numpy as np
import json
from PIL import Image
from torch.utils.data import Dataset, DataLoader
import os    
import torch.nn.functional as F
os.environ['KMP_DUPLICATE_LIB_OK'] = 'True'

In [2]:
class Dataset(Dataset):
    
    def __init__(self, *args): #получаем n массивов данных
        cnt, labels, temp =0, [], []
        self.data = []
        for data in args: #выделяем по одному из них
            self.data.extend(torch.tensor(data).type(torch.float32)) # преобразуем каждый в тензор и добавляем к общим данным
            print(len(data))
            for j in range(len(data)):
                labels.append(cnt) #присовим номера каждому подмассиву для проверки
            cnt += 1
        self.labels = labels
            
    def __getitem__(self, idx):
        return self.data[idx], self.labels[idx]
    
    def __len__(self):
        return len(self.data)

In [3]:
f = open('dataset/stop2.txt').readline()
f = json.loads(f)
print(len(f))

686


In [4]:
f = open('dataset/kulaksgat2.txt').readline()
f = json.loads(f)
print(len(f))

883


In [5]:
[1,2,3,4,5,6][:-2]

[1, 2, 3, 4]

In [6]:
data, temp = [],[]
gests = os.listdir('dataset')
for gest in gests:
    data = []
    print(gest)
    f = open('dataset/' + str(gest)).readline()
    f = json.loads(f)
    for i in range (len(f)):
        data.append(f[i])
    temp.append(data)
data = np.array(temp)

kulaksgat2.txt
stop2.txt


/var/folders/05/318hv04n4zg2jytwsxm19nsw0000gn/T/ipykernel_45129/687087349.py:11: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  data = np.array(temp)


In [7]:
dataset = Dataset(*data)
dataloader = DataLoader(dataset, shuffle=True, batch_size = 16)

883
686


In [8]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(42, 200)
        self.fc2 = nn.Linear(200, 200)
        self.fc3 = nn.Linear(200, 32)
        self.fc4 = nn.Linear(32, 2)
    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))
        x = self.fc4(x)
        return x

In [9]:
net = Net()

In [10]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(net.parameters(), lr=0.01)

In [11]:
for epoch in range(10):
    running_loss = 0.0
    for samples, labels in dataloader:
        #print(samples,labels)
        pred = net(samples)
        loss = criterion(pred, labels)
        
        
        loss.backward()
        optimizer.step()
        
        optimizer.zero_grad()
        running_loss += loss.item()
    print(running_loss)
print('Finished Training')

12.337959557715635
0.0007921106121671073
0.0006690958229569333
0.0005548262881234223
0.0004605078979693644
0.0003989845144189985
0.00032010758990708155
0.0002704216806819204
0.00022861767585169446
0.00019578399484743159
Finished Training


In [12]:
kk=torch.tensor([0.41655007004737854, 0.7089771032333374, 0.4633732736110687, 0.6954310536384583, 0.5125848650932312, 0.6476010084152222, 0.5015125274658203, 0.592243492603302, 0.4613598585128784, 0.5826143026351929, 0.5011842250823975, 0.5836077928543091, 0.509346067905426, 0.5467285513877869, 0.49322056770324707, 0.5981612205505371, 0.48973703384399414, 0.6084542870521545, 0.47082626819610596, 0.5737633109092712, 0.47700440883636475, 0.5433168411254883, 0.46477067470550537, 0.6013384461402893, 0.46333837509155273, 0.599715530872345, 0.4406990706920624, 0.571521520614624, 0.44218024611473083, 0.5451012849807739, 0.436707079410553, 0.6007078289985657, 0.4390093982219696, 0.6013930439949036, 0.4080536961555481, 0.5698967576026917, 0.4114087224006653, 0.549913763999939, 0.41262519359588623, 0.5887411832809448, 0.41415098309516907, 0.5954364538192749])

In [13]:
kk

tensor([0.4166, 0.7090, 0.4634, 0.6954, 0.5126, 0.6476, 0.5015, 0.5922, 0.4614,
        0.5826, 0.5012, 0.5836, 0.5093, 0.5467, 0.4932, 0.5982, 0.4897, 0.6085,
        0.4708, 0.5738, 0.4770, 0.5433, 0.4648, 0.6013, 0.4633, 0.5997, 0.4407,
        0.5715, 0.4422, 0.5451, 0.4367, 0.6007, 0.4390, 0.6014, 0.4081, 0.5699,
        0.4114, 0.5499, 0.4126, 0.5887, 0.4142, 0.5954])

In [16]:
net(kk)

tensor([ 9.1436, -9.2897], grad_fn=<AddBackward0>)

In [18]:
for i in range(10000):
    net(kk)